#### **Welcome to Assignment 3 on Deep Learning for Computer Vision.**
In this assignment you will get a chance to implement basic Visualizations techniques, like- Vanilla Backprop, Integrated Gradient.

#### **Instructions**
1. Use Python 3.x to run this notebook
3. Write your code only in between the lines 'YOUR CODE STARTS HERE' and 'YOUR CODE ENDS HERE'.
you sould not change anything else code cells, if you do, the answers you are supposed to get at the end of this assignment might be wrong.
4. Read documentation of each function carefully.

### Question 1
Visualize saliency map from Pretrained Vgg19 model of a given image using "simple Gradient" method. Find out the mean and maximum  pixel intensity value of the generated saliency map. Note that the input image has 3 channels. To derive a single class saliency value for each pixel (i, j),  take the maximum magnitude across all colour channels.

which of the following values are true?


1.   Mean - 0.0117; Maximum - 0.7853
2.   Mean - 0.0173; Maximum - 0.6645
3.   Mean - 0.0279; Maximum - 0.5638
4.   Mean - 0.0378; Maximum - 0.4726





In [ ]:
#IMPORTS

import torch
import torchvision
import torchvision.transforms as T
import numpy as np
import matplotlib.pyplot as plt
from torchsummary import summary
from PIL import Image

#Using VGG-19 pretrained model for image classification

model = torchvision.models.vgg19(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

# Opening the image
img = Image.open('/content/input.jpg')

# Preprocess the image
def preprocess(image, size=224):
    transform = T.Compose([
        T.Resize((size,size)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        T.Lambda(lambda x: x[None]),
    ])
    return transform(image)

# preprocess the image
X = preprocess(img)

### YOUR CODE STARTS HERE ###



### YOUR CODE ENDS HERE ###

### Question 2

Compute accumulated Attribution over all the input pixels of a given image towards a class (class id-243) output using Integrated Gradient(IG) Method. Assume baseline or reference image to be an image with all zero pixels. Also consider number of steps in IG approximation as 5 and all intermediate step images should follow linear path from baseline image to original image.

which of the following value is true for the estimation of accumulated attribution over all the input pixels across all the channels?

1.   0.9732
2.   0.6931
3.   0.8646
4.   0.5691



In [ ]:
#IMPORTS

import torch
import torchvision
import torchvision.transforms as T
import numpy as np
import matplotlib.pyplot as plt
from torchsummary import summary
from PIL import Image

# Opening the image
img = Image.open('input.jpg') 

# Preprocess the image
def preprocess(image, size=224):
    transform = T.Compose([
        T.Resize((size,size)),
        T.ToTensor(),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        T.Lambda(lambda x: x[None]),
    ])
    return transform(image)

# preprocess the image
X = preprocess(img)

# we need to find the gradient with respect to the input image, so we need to call requires_grad_ on it
X.requires_grad_()

class IG():
    """
        Compute attribution of classifier's output from each input pixels using 
        Integrated Gradient Methood 
    """
    def __init__(self, model):
        self.model = torchvision.models.vgg19(pretrained=True) 
        self.gradients = None
        # Put model in evaluation mode
        self.model.eval()
        # Hook the first layer to get the gradient
        self.hook_layers()

    def hook_layers(self):
        def hook_function(module, grad_in, grad_out):
            self.gradients = grad_in[0]

        # Register hook to the first layer
        first_layer = list(self.model.features._modules.items())[0][1]
        first_layer.register_backward_hook(hook_function)

    def create_images_on_linear_path(self, input_image, steps):
        ''' Create list of all intermediate step images on a linear path
            conneting baseline image to original input image
        '''
        ### YOUR CODE STARTS HERE
        
        ### YOUR CODE ENDS HERE

    def compute_gradients(self, input_image, target_class):
        ''' Compute gradient of model's target class output w.r.t to all input pixels'''  

        ### YOUR CODE STARTS HERE
        
        ### YOUR CODE ENDS HERE

    def compute_integrated_gradients(self, input_image, target_class, steps):

        ''' Main computation of Integrated Gradient method'''
        # Generate xbar images
        x_list = self.create_images_on_linear_path(input_image, steps)
        # Initialize an iamge composed of zeros
        integrated_grads = np.zeros(input_image.size())
        for x_image in x_list:
            # Generate gradients from xbar images
            single_integrated_grad = self.compute_gradients(x_image, target_class)
            # Add rescaled grads from xbar images
            integrated_grads = integrated_grads + single_integrated_grad/steps
        # [0] to get rid of the first channel (1,3,224,224)
        return integrated_grads[0]

### YOUR CODE STARTS HERE

### YOUR CODE ENDS HERE

## Question 3

For a given input image, Find out the channel index and the importance(weight) score of most important feauture map (out of all the feature maps of final convolution layer of a pretrained ResNet50 model) using Grad-CAM method for the class prediction corresponds to "highest logit score". Note that, Grad-CAM produce final heatmap using the weighted combination of the feature map activations, where weights corresponds to importance score.


1.   channel index - 708 ; importance score - 0.0046
2.   channel index - 569 ; importance score - 0.0039
3.   channel index - 1093 ; importance score - 0.0071
4.   channel index - 211 ; importance score - 0.0009


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Function
from torchvision import models


def process_image(img):
    means = [0.485, 0.456, 0.406]
    stds = [0.229, 0.224, 0.225]

    p_img = img.copy()[:, :, ::-1]
    for i in range(3):
        p_img[:, :, i] = p_img[:, :, i] - means[i]
        p_img[:, :, i] = p_img[:, :, i] / stds[i]
    p_img = np.ascontiguousarray(np.transpose(p_img, (2, 0, 1)))
    p_img = torch.from_numpy(p_img)
    p_img.unsqueeze_(0)
    input = p_img.requires_grad_(True)
    return input

class Feat_Extractor():
    """ register gradients get activations from targetted intermediate layers """
    def __init__(self, model, target_layers):
        self.model = model
        self.target_layers = target_layers
        self.gradients = []

    def save_gradient(self, grad):
        self.gradients.append(grad)

    def __call__(self, x):
        outputs = []
        self.gradients = []
        for name, module in self.model._modules.items():
            x = module(x)
            if name in self.target_layers:
                x.register_hook(self.save_gradient)
                outputs += [x]
        return outputs, x


class Netout():
    """ Get network output through forward pass and get intermediate layer representation
    and gradient computation for targeted intermediate layer  """

    def __init__(self, net, feat_module, target_layers):
        self.model = net
        self.feature_module = feat_module
        self.feature_extractor = Feat_Extractor(self.feature_module, target_layers)

    def get_gradients(self):
        return self.feature_extractor.gradients

    def __call__(self, x):
        target_activations = []
        for name, module in self.model._modules.items():
            if module == self.feature_module:
                target_activations, x = self.feature_extractor(x)

            elif "avgpool" in name.lower():
                x = module(x)
                x = x.view(x.size(0),-1)
    
            else:
                x = module(x)
        
        return target_activations, x


#visualize heatmap on input image
def visualize(img, mask):
    heatmap = cv2.applyColorMap(np.uint8(255 * mask), cv2.COLORMAP_JET)
    heatmap = np.float32(heatmap) / 255
    cam = heatmap + np.float32(img)
    cam = cam / np.max(cam)
    img = np.uint8(255 * cam)
    plt.imshow(img)


class GradientCam:
    def __init__(self, model, feature_module, target_layer_, cuda):
        self.model = model
        self.feature_module = feature_module
        self.model.eval()
        self.cuda = cuda
        if self.cuda:
            self.model = model.cuda()

        self.extractor = Netout(self.model, self.feature_module, target_layer_)

    def forward(self, input):
        return self.model(input)

    def __call__(self, input, index=None):
        
        ''' This function should Return weights corresponding to each feature map of last convolution layer.
        Note that, linear combination of such weights with last conv layer feature map finally 
        produce the explanation map'''

        ### YOUR CODE STARTS HERE
        
        
        ### YOUR CODE ENDS HERE


if __name__ == '__main__':
    
    image = cv2.imread('/content/input.jpg', 1)
    image = np.float32(cv2.resize(image, (224, 224))) / 255
    input = process_image(image)

    net = models.resnet50(pretrained=True)
    grad_cam = GradientCam(model=net, feature_module=net.layer4, \
                       target_layer_=["2"], cuda=False)
    

    target_index = None

    ### call to grad_cam method should return the importance vector corresponds to
    ### each feature map of last convolution layer of pretrained Resnet50

    weights, class_activation_map = grad_cam(input, target_index)

    print ("Most important feature map index: ", np.argmax(weights))
    print ("Its corresponding importance is: ", np.max(weights))

    visualize(image, class_activation_map)